In [1]:
import pathlib
import collections
import math
import numpy as np
import itertools

In [2]:
testlines = pathlib.Path('day19sample.txt').read_text().splitlines()

In [3]:
puzzlelines = pathlib.Path('day19.txt').read_text().splitlines()

## part 1 ##

In [4]:
def parse(lines):
    scanners = collections.defaultdict(list)
    for line in lines:
        if '--' in line:
            _, _, num, _ = line.split()
            curr_scanner = int(num)
        elif line:
            x, y, z = line.split(',')
            scanners[curr_scanner].append((int(x), int(y), int(z)))
        else:
            continue
    return scanners

In [5]:
def distance(pt1, pt2):
    return sum(abs(pt1[i]-pt2[i]) for i in range(3))

In [6]:
def get_all_pair_distances(pts):
    numpts = len(pts)
    d = collections.defaultdict(dict)
    for i in range(numpts-1):
        ipt = pts[i]
        for j in range(i+1, numpts):
            jpt = pts[j]
            distij = distance(ipt, jpt)
            d[ipt][jpt] = distij
            d[jpt][ipt] = distij
    return d        

In [7]:
def get_nearest_neighbor_distances(pairdict):
    nndists = set()
    for pt in pairdict:
        nndists.add(min(pairdict[pt].values()))
    return nndists

In [8]:
def build_nearest_neighborlist(scannerdists):
    nlist = {}
    for scanner, pairdict in scannerdists.items():
        nlist[scanner] = get_nearest_neighbor_distances(pairdict)
    return nlist

In [9]:
def find_overlapping_scanners(nlist):
    overlaps = {}
    for i in nlist:
        curr_overlaps = {}
        for j in nlist:
            if j == i:
                continue
            curr_overlaps[j] = len(set.intersection(nlist[i],nlist[j]))
        overlaps[i] = curr_overlaps
    return overlaps

In [10]:
def find_unique_beacons(scanners):
    unique = set()
    for scanner in scanners:
        unique |= set([beacon for beacon in scanners[scanner]])
    return unique

In [11]:
def find_reference_points(distances, shared):
    '''Find a shared beacon that has two neighbors with shared distances to it, and return all 3'''
    for firstpt, d in distances.items():
        refpts = [firstpt]
        for pt, dist in d.items():
            if dist in shared:
                refpts.append(pt)
        if len(refpts) >= 3:
            return refpts
    return None

In [12]:
def match_reference_points(refpts, sda, sdb):
    origina = refpts[0]
    joined_dists = [sda[origina][pt] for pt in refpts[1:3]]
    for firstpt, d in sdb.items():
        if set(joined_dists) <= set(d.values()):
            # found the origin point in b
            refptsb = [firstpt]
            for secondpt, dist in d.items():
                if dist == joined_dists[0]:
                    refptsb.append(secondpt)
                    break
            for thirdpt, dist in d.items():
                if dist == joined_dists[1]:
                    refptsb.append(thirdpt)
                    break
            return refptsb
    return None

In [13]:
def translate(pts, new_origin):
    x0, y0, z0 = new_origin
    newpts = []
    for x,y,z in pts:
        newpts.append((x-x0, y-y0, z-z0))
    return newpts

In [14]:
def get_axis_map(pta, ptb):
    axismap = {}
    for i,vala in enumerate(pta):
        for j,valb in enumerate(ptb):
            if vala == valb:
                axismap[i] = (j, +1)
                break
            elif vala == -valb:
                axismap[i] = (j, -1)
                break
    return axismap            

In [15]:
def reorient(axismap, pts):
    newpts = []
    for pt in pts:
        newpt = [0,0,0]
        for idx, (newidx,sgn) in axismap.items():
            newpt[idx] = pt[newidx]*sgn
        newpts.append(tuple(newpt))
    return newpts

In [16]:
def reconcile(a, b, shared, scannerdists, scanners):
    refptsa = find_reference_points(scannerdists[a], shared)
    if refptsa is None:
        return False
    refptsb = match_reference_points(refptsa, scannerdists[a], scannerdists[b])
    if refptsb is None:
        return False
    shiftrefa = translate(refptsa, refptsa[0])
    shiftrefb = translate(refptsb, refptsb[0])
    axismap = get_axis_map(shiftrefa[1], shiftrefb[1])
    shiftb = translate(scanners[b], refptsb[0])
    newb = reorient(axismap, shiftb)
    newb = translate(newb, tuple(-val for val in refptsa[0]))
    scanners[b] = newb
    scannerdists[b] = get_all_pair_distances(scanners[b])
    return True

In [17]:
def solve(lines):
    scanners = parse(lines)
    scannerdists = {}
    for scanner,pts in scanners.items():
        scannerdists[scanner] = get_all_pair_distances(pts)
    nlist = build_nearest_neighborlist(scannerdists)
    overlaps = find_overlapping_scanners(nlist)
    scannercombs = itertools.combinations(range(len(overlaps)), 2)
    reconciled = set([0]) # list of scanners that have finished coordinates
    allscanners = set(scanners.keys())
    i = 0
    while reconciled < allscanners and i < 100:
        scannercombs = itertools.product(reconciled, allscanners - reconciled)
        for a, b in scannercombs:
            if b in reconciled:
                continue
            if overlaps[a][b] >= 5:
                shared = set.intersection(nlist[a], nlist[b])
                success = reconcile(a, b, shared, scannerdists, scanners)
                if success:
                    reconciled.add(b)
        i += 1
    print(sorted(reconciled))
    unique = find_unique_beacons(scanners)
    return len(unique)

In [18]:
solve(testlines)

[0, 1, 2, 3, 4]


79

In [19]:
solve(puzzlelines)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]


454

## part 2 ##

In [20]:
def reconcile2(a, b, shared, scannerdists, scanners, scannerpositions):
    refptsa = find_reference_points(scannerdists[a], shared)
    if refptsa is None:
        return False
    refptsb = match_reference_points(refptsa, scannerdists[a], scannerdists[b])
    if refptsb is None:
        return False
    shiftrefa = translate(refptsa, refptsa[0])
    shiftrefb = translate(refptsb, refptsb[0])
    axismap = get_axis_map(shiftrefa[1], shiftrefb[1])
    shiftb = translate(scanners[b], refptsb[0])
    newb = reorient(axismap, shiftb)
    newb = translate(newb, tuple(-val for val in refptsa[0]))
    scanners[b] = newb
    scannerdists[b] = get_all_pair_distances(scanners[b])
    scannerpos = scannerpositions[b]
    shiftb = translate([scannerpos], refptsb[0])
    newb = reorient(axismap, shiftb)
    newb = translate(newb, tuple(-val for val in refptsa[0]))
    scannerpositions[b] = newb[0]
    return True

In [31]:
def solve2(lines):
    scanners = parse(lines)
    scannerpositions = [(0,0,0) for scanner in scanners]
    scannerdists = {}
    for scanner,pts in scanners.items():
        scannerdists[scanner] = get_all_pair_distances(pts)
    nlist = build_nearest_neighborlist(scannerdists)
    overlaps = find_overlapping_scanners(nlist)
    scannercombs = itertools.combinations(range(len(overlaps)), 2)
    reconciled = set([0]) # list of scanners that have finished coordinates
    allscanners = set(scanners.keys())
    i = 0
    while reconciled < allscanners and i < 100:
        scannercombs = itertools.product(reconciled, allscanners - reconciled)
        for a, b in scannercombs:
            if b in reconciled:
                continue
            if overlaps[a][b] >= 5:
                shared = set.intersection(nlist[a], nlist[b])
                success = reconcile2(a, b, shared, scannerdists, scanners, scannerpositions)
                if success:
                    reconciled.add(b)
        i += 1
    print(sorted(reconciled))
    unique = find_unique_beacons(scanners)
    num_unique = len(unique)
    scannerposdistances = get_all_pair_distances(scannerpositions)
    maxdist = 0
    for pt, d in scannerposdistances.items():
        curr_max = max(d.values())
        if curr_max > maxdist:
            maxdist = curr_max
    return num_unique, maxdist

In [32]:
solve2(testlines)

[0, 1, 2, 3, 4]


(79, 3621)

In [33]:
solve2(puzzlelines)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]


(454, 10813)